In [4]:
!pip install tensorflow gymnasium keras-rl2 gymnasium[Atari] swig stable_baselines3[extra] moviepy

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached keras_rl2-1.0.5-py3-none-any.whl.metadata (304 bytes)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metad

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Kent\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\

In [1]:
import gymnasium as gym
import ale_py
import random
import stable_baselines3

In [53]:
#Create Environment
env = gym.make('SpaceInvaders-v4', render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [55]:
#Check Actions
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [54]:
print(env.observation_space)


Box(0, 255, (210, 160, 3), uint8)


In [ ]:
#Random Agent
episodes = 5

for episode in range(1, episodes + 1):
    state, _ = env.reset()  
    done = False
    score = 0 

    while not done:
        env.render()  
        
        # Sample a random action from the environment's action space
        action = env.action_space.sample()  
        n_state, reward, done, truncated, info = env.step(action)  # Perform the action
        score += reward

    print(f'Episode: {episode} Score: {score}')

env.close()

Episode: 1 Score: 105.0
Episode: 2 Score: 215.0
Episode: 3 Score: 230.0
Episode: 4 Score: 210.0
Episode: 5 Score: 210.0


In [ ]:
#Trains Agent
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import gymnasium as gym
from gymnasium.spaces import Box
import numpy as np

class ChannelFirstWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(ChannelFirstWrapper, self).__init__(env)
        shape = env.observation_space.shape
        self.observation_space = Box(
            low=env.observation_space.low.min(),
            high=env.observation_space.high.max(),
            shape=(shape[2], shape[0], shape[1]),  
            dtype=env.observation_space.dtype
        )

    def observation(self, observation):
        return np.transpose(observation, (2, 0, 1))  

class Float16Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(Float16Wrapper, self).__init__(env)
        self.observation_space = Box(
            low=self.observation_space.low.min(),
            high=self.observation_space.high.max(),
            shape=self.observation_space.shape,
            dtype=np.float16
        )

    def observation(self, observation):
        return observation.astype(np.float16)

env_id = "ALE/SpaceInvaders-v5"
num_envs = 2

env = DummyVecEnv([lambda: Float16Wrapper(ChannelFirstWrapper(gym.make(env_id))) for _ in range(num_envs)])

env = VecFrameStack(env, n_stack=4)

policy_kwargs = dict(normalize_images=False)
model = PPO(
    "CnnPolicy", env,
    verbose=1,
    n_steps=512,
    batch_size=64,
    learning_rate=0.0001,
    gamma=0.99,
    tensorboard_log="ppo_logs",
    policy_kwargs=policy_kwargs
)

model.learn(total_timesteps=1000000)

model.save("ppo_spaceinvaders_optimized")


Using cpu device
Logging to ppo_logs\PPO_17
-----------------------------
| time/              |      |
|    fps             | 75   |
|    iterations      | 1    |
|    time_elapsed    | 13   |
|    total_timesteps | 1024 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 11          |
|    iterations           | 2           |
|    time_elapsed         | 172         |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.017209377 |
|    clip_fraction        | 0.313       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.73       |
|    explained_variance   | 0.012764096 |
|    learning_rate        | 0.0001      |
|    loss                 | 7.39        |
|    n_updates            | 10          |
|    policy_gradient_loss | 0.0124      |
|    value_loss           | 5.48e+03    |
--------------------------------

In [ ]:
#Tests and Renders Video
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import time
from stable_baselines3.common.vec_env import VecVideoRecorder
import cv2
import numpy as np

def make_env(env_id, rank, render_mode="rgb_array"):  
    def _init():
        env = gym.make(env_id, render_mode=render_mode)
        env.reset(seed=rank) 
        return env
    return _init

def reshape_obs(obs):
    """
    Ensure the observation has the shape (n_envs, C, H, W).
    If the observation is in shape (1, 210, 160, 12), resize it to (3, 210, 640).
    """
    if obs.shape == (1, 210, 160, 12):  
        obs = obs[:, :, :, :3] 
        

        obs_resized = np.zeros((1, 210, 640, 3), dtype=np.uint8)
        for i in range(obs.shape[0]):
            obs_resized[i] = cv2.resize(obs[i], (640, 210)) 
        
        obs = np.transpose(obs_resized, (0, 3, 1, 2))  
        
    return obs

if __name__ == "__main__":
    env_id = "ALE/SpaceInvaders-v5"
    num_envs = 1 

    env = DummyVecEnv([make_env(env_id, i) for i in range(num_envs)])

    env = VecFrameStack(env, n_stack=4)

    video_folder = "videos"
    env = VecVideoRecorder(env, video_folder, record_video_trigger=lambda x: x == 0, video_length=500000, name_prefix="spaceinvaders")

    model_save_path = "ppo_spaceinvaders_optimized" 
    model = PPO.load(model_save_path)

    obs = env.reset()
    done = False
    total_reward = 0

    while not done:
        obs = reshape_obs(obs)
        
        action, _ = model.predict(obs)

        obs, reward, done, info = env.step(action)

        total_reward += reward

        env.render()

        time.sleep(0.05)

    print(f"Total Reward: {total_reward}")

    env.close()|


Total Reward: [165.]
MoviePy - Building video C:\Users\Kent\Desktop\AI Proj\videos\spaceinvaders-step-0-to-step-500000.mp4.
MoviePy - Writing video C:\Users\Kent\Desktop\AI Proj\videos\spaceinvaders-step-0-to-step-500000.mp4



MoviePy - Done !
MoviePy - video ready C:\Users\Kent\Desktop\AI Proj\videos\spaceinvaders-step-0-to-step-500000.mp4
